In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from General_Functions import long_horizon_ret
from Main import settings, features, pf_set
from pandas.tseries.offsets import MonthEnd
from Prepare_Data import process_risk_free_rate, process_return_data, wealth_func, load_and_filter_market_returns_test, process_all_data, process_cluster_labels
from General_Functions import size_screen_fun, addition_deletion_fun
from Estimate_Covariance_Matrix import process_cluster_data
import numpy as np
from scipy.stats import rankdata
import re
import gc
import ewma

import statsmodels.formula.api as smf

Features list length: 154
All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%


ValueError: too many values to unpack (expected 3)

In [4]:
#Indhenter chars og daily:
file_path_usa_test = "./data_test/usa_test.parquet"
daily_file_path = "./data_test/usa_dsf_test.parquet"
file_path_world_ret = "./data_test/world_ret_test.csv"
risk_free_path = "./data_test/risk_free_test.csv"
market_path = "./data_test/market_returns_test.csv"

# Kald funktionen
chars, daily = process_all_data(file_path_usa_test, daily_file_path, file_path_world_ret, risk_free_path, market_path)
print(chars)
print(daily)

All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%
          id        eom   sic size_grp            me  rvol_252d  dolvol_126d  \
1      10104 2010-02-28  7372     mega  123619.74809   0.343023     0.854651   
2      10104 2010-03-31  7372     mega  129040.82501   0.267442     0.854651   
3      10104 2010-04-30  7372     mega  130101.18774   0.284884     0.848837   
4      10104 2010-05-31  7372     mega  113436.81847   0.354651     0.848837   
5      10104 2010-06-30  7372     mega  107857.95540   0.261628     0.843023   
...      ...        ...   ...      ...           ...        ...          ...   
28707  92655 2023-06-30  6324     mega  447491.23412   0.284884     0.953488   
28708  92655 2023-07-31  6324     mega  471446.66929   0.319767     0.953488   
28709  92655 2023-08-31  6324     mega  441458.42446   0.348837     0.941860   
28710  926

## Prepare cluster characteristics ---------------

In [5]:
file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"
cluster_labels = process_cluster_labels(file_path_cluster_labels, file_path_factor_details)
print(cluster_labels)

       characteristic       cluster  direction
0                 age  low_leverage       -1.0
1             aliq_at    investment       -1.0
2            aliq_mat  low_leverage       -1.0
3            ami_126d          size        1.0
4               at_be  low_leverage       -1.0
..                ...           ...        ...
149           z_score  low_leverage        1.0
150  zero_trades_126d      low_risk        1.0
151   zero_trades_21d      low_risk        1.0
152  zero_trades_252d      low_risk        1.0
153         rvol_252d      low_risk       -1.0

[154 rows x 3 columns]


In [6]:
# Prepare cluster characteristics
valid_chars = chars[chars["valid"] == True]
cluster_data_m = valid_chars[["id", "eom", "size_grp", "ff12"] + features].copy()

clusters = cluster_labels["cluster"].unique()
cluster_ranks = {}

for cl in clusters:
    chars_sub = cluster_labels[(cluster_labels["cluster"] == cl) & (cluster_labels["characteristic"].isin(features))]

    data_sub = cluster_data_m[chars_sub["characteristic"].values].copy()

    for c in chars_sub["characteristic"].values:
        dir_value = chars_sub.loc[chars_sub["characteristic"] == c, "direction"].values[0]
        if dir_value == -1:
            data_sub[c] = 1 - data_sub[c]

    cluster_ranks[cl] = data_sub.mean(axis=1)

# Konverter cluster ranks til en dataframe
cluster_ranks_df = pd.DataFrame(cluster_ranks)

# Kombiner med de oprindelige data
cluster_data_m = cluster_data_m[["id", "eom", "size_grp", "ff12"]].copy()
#cluster_data_m["eom_ret"] = cluster_data_m["eom"] + pd.DateOffset(months=1) - pd.DateOffset(days=1) #Ikke helt korrekt
cluster_data_m["eom_ret"] = cluster_data_m["eom"] + pd.DateOffset(months=1)
cluster_data_m["eom_ret"] = cluster_data_m["eom_ret"] + pd.offsets.MonthEnd(0)  # Sikrer, at det er den sidste dag i måneden



# Merge cluster rankings
cluster_data_m = pd.concat([cluster_data_m, cluster_ranks_df], axis=1)

# Output result
print(cluster_data_m)

          id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
24     10104 2012-01-31     mega  BusEq 2012-02-29      0.763518    0.497303   
25     10104 2012-02-29     mega  BusEq 2012-03-31      0.775977    0.498864   
26     10104 2012-03-31     mega  BusEq 2012-04-30      0.772860    0.593682   
27     10104 2012-04-30     mega  BusEq 2012-05-31      0.723207    0.556345   
28     10104 2012-05-31     mega  BusEq 2012-06-30      0.742939    0.566149   
...      ...        ...      ...    ...        ...           ...         ...   
28707  92655 2023-06-30     mega  Money 2023-07-31      0.525670    0.231943   
28708  92655 2023-07-31     mega  Money 2023-08-31      0.469914    0.189911   
28709  92655 2023-08-31     mega  Money 2023-09-30      0.519169    0.187129   
28710  92655 2023-09-30     mega  Money 2023-10-31      0.506901    0.200052   
28711  92655 2023-10-31     mega  Money 2023-11-30      0.511762    0.223770   

           size     value   quality  lo

In [7]:
# Add Industry/market dummies
if settings["cov_set"]["industries"]:
    # Opret industry dummies
    industries = sorted(cluster_data_m["ff12"].unique())
    for ind in industries:
        cluster_data_m[str(ind)] = (cluster_data_m["ff12"] == ind).astype(int)
    ind_factors = industries
else:
    # Tilføj markedsfaktor
    cluster_data_m["mkt"] = 1
    ind_factors = ["mkt"]

# Output test
print(cluster_data_m)

          id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
24     10104 2012-01-31     mega  BusEq 2012-02-29      0.763518    0.497303   
25     10104 2012-02-29     mega  BusEq 2012-03-31      0.775977    0.498864   
26     10104 2012-03-31     mega  BusEq 2012-04-30      0.772860    0.593682   
27     10104 2012-04-30     mega  BusEq 2012-05-31      0.723207    0.556345   
28     10104 2012-05-31     mega  BusEq 2012-06-30      0.742939    0.566149   
...      ...        ...      ...    ...        ...           ...         ...   
28707  92655 2023-06-30     mega  Money 2023-07-31      0.525670    0.231943   
28708  92655 2023-07-31     mega  Money 2023-08-31      0.469914    0.189911   
28709  92655 2023-08-31     mega  Money 2023-09-30      0.519169    0.187129   
28710  92655 2023-09-30     mega  Money 2023-10-31      0.506901    0.200052   
28711  92655 2023-10-31     mega  Money 2023-11-30      0.511762    0.223770   

           size     value   quality  ..

In [8]:
# Standardiser faktorer (clusters) per eom
cluster_data_m[clusters] = cluster_data_m.groupby("eom")[clusters].transform(lambda x: (x - x.mean()) / x.std())
print(cluster_data_m)

          id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
24     10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
25     10104 2012-02-29     mega  BusEq 2012-03-31      1.773561   -0.005543   
26     10104 2012-03-31     mega  BusEq 2012-04-30      1.774189    0.553260   
27     10104 2012-04-30     mega  BusEq 2012-05-31      1.422269    0.312431   
28     10104 2012-05-31     mega  BusEq 2012-06-30      1.560002    0.372443   
...      ...        ...      ...    ...        ...           ...         ...   
28707  92655 2023-06-30     mega  Money 2023-07-31      0.192269   -1.748730   
28708  92655 2023-07-31     mega  Money 2023-08-31     -0.216928   -2.017339   
28709  92655 2023-08-31     mega  Money 2023-09-30      0.147315   -2.030984   
28710  92655 2023-09-30     mega  Money 2023-10-31      0.052639   -1.975682   
28711  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   

           size     value   quality  ..

In [9]:
unique_dates = daily["eom"].unique()
print(unique_dates[20:50])

<DatetimeArray>
['2011-10-31 00:00:00', '2011-11-30 00:00:00', '2011-12-31 00:00:00',
 '2012-01-31 00:00:00', '2012-02-29 00:00:00', '2012-03-31 00:00:00',
 '2012-04-30 00:00:00', '2012-05-31 00:00:00', '2012-06-30 00:00:00',
 '2012-07-31 00:00:00', '2012-08-31 00:00:00', '2012-09-30 00:00:00',
 '2012-10-31 00:00:00', '2012-11-30 00:00:00', '2012-12-31 00:00:00',
 '2013-01-31 00:00:00', '2013-02-28 00:00:00', '2013-03-31 00:00:00',
 '2013-04-30 00:00:00', '2013-05-31 00:00:00', '2013-06-30 00:00:00',
 '2013-07-31 00:00:00', '2013-08-31 00:00:00', '2013-09-30 00:00:00',
 '2013-10-31 00:00:00', '2013-11-30 00:00:00', '2013-12-31 00:00:00',
 '2014-01-31 00:00:00', '2014-02-28 00:00:00', '2014-03-31 00:00:00']
Length: 30, dtype: datetime64[ns]


In [10]:
# Tilføj daglige returdata
min_eom = cluster_data_m["eom"].min()
daily_filtered = daily[daily["date"] >= min_eom][["id", "date", "ret_exc", "eom"]].copy()
daily_filtered.rename(columns={"eom": "eom_ret"}, inplace=True)

# Flet daglige data med cluster_data_m
cluster_data_d = cluster_data_m.merge(daily_filtered, on=["id", "eom_ret"], how="inner")

print(cluster_data_d)
print(cluster_data_d.shape)
# Fjern rækker med manglende værdier
cluster_data_d.dropna(inplace=True)

print(cluster_data_d)
print(cluster_data_d.shape)

           id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
0       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
1       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
2       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
3       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
4       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
...       ...        ...      ...    ...        ...           ...         ...   
511495  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   
511496  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   
511497  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   
511498  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   
511499  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   

            size     value 

#### Now in a function

In [11]:
def process_cluster_data(chars, daily, cluster_labels_path, factor_details_path):
    """
    Behandler cluster data ved at læse og forberede cluster labels, beregne cluster ranks,
    tilføje industry/market dummies, standardisere faktorer og flette daglige returdata.

    Parametre:
    - chars: DataFrame med karakteristika for aktier
    - daily: DataFrame med daglige afkastdata
    - cluster_labels_path: Sti til CSV-filen med cluster labels
    - factor_details_path: Sti til Excel-filen med factor details

    Returnerer:
    - cluster_data_d: Behandlet DataFrame med alle transformationer

    Eksempel på brug:
    file_path_cluster_labels = "Data/Cluster Labels.csv"
    file_path_factor_details = "Data/Factor Details.xlsx"
    cluster_data_d, ind_factors, clusters = process_cluster_data(chars, daily, file_path_cluster_labels, file_path_factor_details)
    """

    # Indlæs cluster labels separat (bruger samme metode som i den anden version)
    cluster_labels = process_cluster_labels(cluster_labels_path, factor_details_path)

    # Filtrer gyldige karakteristika
    valid_chars = chars[chars["valid"] == True]
    cluster_data_m = valid_chars[["id", "eom", "size_grp", "ff12"] + features].copy()

    # Beregn cluster ranks
    clusters = cluster_labels["cluster"].unique()
    cluster_ranks = {}

    for cl in clusters:
        chars_sub = cluster_labels[(cluster_labels["cluster"] == cl) & (cluster_labels["characteristic"].isin(features))]

        # Undgå potentielle fejl ved at sikre, at kun eksisterende kolonner vælges
        valid_features = [c for c in chars_sub["characteristic"].values if c in cluster_data_m.columns]
        data_sub = cluster_data_m[valid_features].copy()

        for c in valid_features:
            dir_value = chars_sub.loc[chars_sub["characteristic"] == c, "direction"].values[0]
            if dir_value == -1:
                data_sub[c] = 1 - data_sub[c]

        cluster_ranks[cl] = data_sub.mean(axis=1)

    # Konverter cluster ranks til DataFrame
    cluster_ranks_df = pd.DataFrame(cluster_ranks)

    # Kombiner med de oprindelige data
    cluster_data_m = cluster_data_m[["id", "eom", "size_grp", "ff12"]].copy()
    cluster_data_m["eom_ret"] = cluster_data_m["eom"] + pd.DateOffset(months=1)
    cluster_data_m["eom_ret"] = cluster_data_m["eom_ret"] + pd.offsets.MonthEnd(0)  # Sikrer, at det er sidste dag i måneden

    # Merge cluster rankings
    cluster_data_m = pd.concat([cluster_data_m, cluster_ranks_df], axis=1)

    # Tilføj industry/market dummies
    if settings["cov_set"]["industries"]:
        industries = sorted(cluster_data_m["ff12"].unique())
        for ind in industries:
            cluster_data_m[str(ind)] = (cluster_data_m["ff12"] == ind).astype(int)
        ind_factors = industries # vi bruger ik den her?
    else:
        cluster_data_m["mkt"] = 1
        ind_factors = ["mkt"] # vi bruger ik den her?

    # Standardiser faktorer per eom (Bruger samme metode som den anden version)
    cluster_data_m[clusters] = cluster_data_m.groupby("eom")[clusters].transform(lambda x: (x - x.mean()) / x.std())

    # Indlæs daglige returdata
    min_eom = cluster_data_m["eom"].min()
    daily_filtered = daily[daily["date"] >= min_eom][["id", "date", "ret_exc", "eom"]].copy()
    daily_filtered.rename(columns={"eom": "eom_ret"}, inplace=True)

    # Flet daglige data med cluster_data_m
    cluster_data_d = cluster_data_m.merge(daily_filtered, on=["id", "eom_ret"], how="inner")

    # Fjern manglende værdier
    cluster_data_d.dropna(inplace=True)

    return cluster_data_d, ind_factors, clusters

In [12]:
ile_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"
cluster_data_d, ind_factors, clusters = process_cluster_data(chars, daily, file_path_cluster_labels, file_path_factor_details)


In [13]:
type(cluster_data_d)

pandas.core.frame.DataFrame

In [14]:
print(type(cluster_data_d))


<class 'pandas.core.frame.DataFrame'>


## Create factor returns --------------------

In [15]:
cluster_data_d

,id,eom,size_grp,ff12,eom_ret,low_leverage,investment,size,value,quality,...,Hlth,Manuf,Money,NoDur,Other,Shops,Telcm,Utils,date,ret_exc
0,10104,2012-01-31,mega,BusEq,2012-02-29,1.680821,-0.014307,-0.994839,-0.731867,0.806993,...,0,0,0,0,0,0,0,0,2012-02-01,0.023749
1,10104,2012-01-31,mega,BusEq,2012-02-29,1.680821,-0.014307,-0.994839,-0.731867,0.806993,...,0,0,0,0,0,0,0,0,2012-02-02,-0.002425
2,10104,2012-01-31,mega,BusEq,2012-02-29,1.680821,-0.014307,-0.994839,-0.731867,0.806993,...,0,0,0,0,0,0,0,0,2012-02-03,0.010412
3,10104,2012-01-31,mega,BusEq,2012-02-29,1.680821,-0.014307,-0.994839,-0.731867,0.806993,...,0,0,0,0,0,0,0,0,2012-02-06,-0.003780
4,10104,2012-01-31,mega,BusEq,2012-02-29,1.680821,-0.014307,-0.994839,-0.731867,0.806993,...,0,0,0,0,0,0,0,0,2012-02-07,-0.001726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511495,92655,2023-10-31,mega,Money,2023-11-30,0.091804,-1.820302,-1.821256,-0.440187,0.912515,...,0,0,1,0,0,0,0,0,2023-11-24,0.005932
511496,92655,2023-10-31,mega,Money,2023-11-30,0.091804,-1.820302,-1.821256,-0.440187,0.912515,...,0,0,1,0,0,0,0,0,2023-11-27,-0.006626
511497,92655,2023-10-31,mega,Money,2023-11-30,0.091804,-1.820302,-1.821256,-0.440187,0.912515,...,0,0,1,0,0,0,0,0,2023-11-28,-0.005840
511498,92655,2023-10-31,mega,Money,2023-11-30,0.091804,-1.820302,-1.821256,-0.440187,0.912515,...,0,0,1,0,0,0,0,0,2023-11-29,-0.010478


In [16]:
def run_regressions_by_date(cluster_data_d, ind_factors, clusters):
    """
    Kører en lineær regression pr. dato på 'cluster_data_d' og returnerer en DataFrame med resultaterne.

    Parametre:
      - cluster_data_d: DataFrame med mindst kolonnerne 'date' og 'ret_exc', samt de forklarende variable
      - ind_factors: Liste med navne på industry/market faktorer (f.eks. ['BusEq', 'Chems', ...])
      - clusters: Liste med navne på cluster faktorer (f.eks. ['low_leverage', 'investment', ...])
    
    Returnerer:
      En DataFrame med en række per dato, indeholdende:
        - date: Datoen for gruppen
        - data: Under-DataFrame med de data, der blev brugt for den dato
        - fit: Det tilpassede regressionsmodel-objekt (statsmodels)
        - res: Residualerne fra modellen
        - tidied: En "tidy" tabel med regressionsresultater (koefficienter, standardfejl, t-værdier, p-værdier mv.)
    Example:
    fct_ret_est = run_regressions_by_date(cluster_data_d, ind_factors, clusters)
        
    """
    # Kombiner de forklarende variable
    all_factors = list(ind_factors) + list(clusters)
    formula = "ret_exc ~ -1 + " + " + ".join(all_factors)
    
    results = []
    # Gruppér data efter 'date'
    for date, group in cluster_data_d.groupby('date'):
        # Kør regressionen uden intercept (-1)
        model = smf.ols(formula=formula, data=group).fit()
        # Udtræk residualer
        residuals = model.resid
        # Opret en "tidy" oversigt over regressionsresultaterne vha. summary2
        tidy_df = model.summary2().tables[1].reset_index().rename(columns={'index': 'term'})
        
        results.append({
            'date': date,
            'data': group,
            'fit': model,
            'res': residuals,
            'tidied': tidy_df
        })

    fct_ret_est = pd.DataFrame(results)
    return fct_ret_est

In [17]:
fct_ret_est = run_regressions_by_date(cluster_data_d, ind_factors, clusters)

In [18]:
fct_ret_est

,date,data,fit,res,tidied
0,2012-02-01,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,0 0.013814 2978 0.004703 5956 ...,term Coef. Std.Err. ...
1,2012-02-02,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,1 -0.003386 2979 0.007781 5957 ...,term Coef. Std.Err. ...
2,2012-02-03,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,2 0.003454 2980 -0.012202 5958 ...,term Coef. Std.Err. ...
3,2012-02-06,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,3 -0.000870 2981 0.004308 5959 ...,term Coef. Std.Err. ...
4,2012-02-07,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,4 0.002190 2982 0.003575 5960 ...,term Coef. Std.Err. ...
...,...,...,...,...,...
2973,2023-11-24,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,2973 -0.002252 5951 0.003634 8929 ...,term Coef. Std.Err. ...
2974,2023-11-27,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,2974 0.006810 5952 0.000513 8930 ...,term Coef. Std.Err. ...
2975,2023-11-28,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,2975 0.002065 5953 0.015343 8931 ...,term Coef. Std.Err. ...
2976,2023-11-29,id eom size_grp ff12 eo...,<statsmodels.regression.linear_model.Regressio...,2976 0.003734 5954 -0.013097 8932 ...,term Coef. Std.Err. ...


In [19]:
def create_factor_returns(fct_ret_est):
    """
    Omformer fct_ret_est til et datasæt med faktorafkast på samme måde som R-koden:
      fct_ret <- fct_ret_est %>%
                   unnest(tidied) %>%
                   select(date, term, estimate) %>%
                   pivot_wider(names_from = term, values_from = estimate) %>%
                   arrange(date) %>%
                   setDT()
                   
    For hver række i fct_ret_est:
      - "Unnestes" den nestede DataFrame i kolonnen 'tidied'
      - Kolonnen med koefficienter, som i vores tilfælde hedder "Coef.", omdøbes til "estimate"
      - 'date' tilføjes fra den overordnede række
      - Kun kolonnerne 'date', 'term' og 'estimate' bevares
    Herefter pivotteres dataene, så hver unik værdi i 'term' bliver en kolonne, og vi sorterer efter 'date'.
    
    Parametre:
      - fct_ret_est: DataFrame med mindst kolonnerne 'date' og 'tidied', hvor 'tidied' er en nestet DataFrame
      
    Returnerer:
      - fct_ret: Pivotteret DataFrame med én række pr. dato og en kolonne pr. term, indeholdende faktorafkast (estimate)
      
    Example:
    fct_ret = create_factor_returns(fct_ret_est)
    """
    tidied_list = []
    # Gå igennem hver række i fct_ret_est
    for _, row in fct_ret_est.iterrows():
        # Kopiér den nestede DataFrame
        tidy_df = row['tidied'].copy()
        # Hvis kolonnen hedder "Coef.", omdøb den til "estimate"
        if 'Coef.' in tidy_df.columns:
            tidy_df = tidy_df.rename(columns={'Coef.': 'estimate'})
        # Tilføj datoen fra den overordnede række
        tidy_df['date'] = row['date']
        # Vælg kun de ønskede kolonner
        tidied_list.append(tidy_df[['date', 'term', 'estimate']])
    
    # Sammensæt alle de unnestede DataFrames
    combined = pd.concat(tidied_list, ignore_index=True)
    
    # Pivotér dataene: 'date' bliver indeks, 'term' bliver kolonnenavne, og cellerne udfyldes med 'estimate'
    fct_ret = combined.pivot(index='date', columns='term', values='estimate').reset_index()
    
    # Sortér efter 'date'
    fct_ret = fct_ret.sort_values('date').reset_index(drop=True)
    
    return fct_ret


In [20]:
fct_ret = create_factor_returns(fct_ret_est)

In [21]:
fct_ret

term,date,BusEq,Chems,Durbl,Enrgy,Hlth,Manuf,Money,NoDur,Other,...,low_leverage,low_risk,momentum,profit_growth,profitability,quality,seasonality,short_term_reversal,size,value
0,2012-02-01,0.015017,0.008651,-0.010546,0.003255,0.012807,0.014745,0.018380,0.005752,0.011182,...,0.000192,-0.001348,-0.000122,0.000656,-0.002363,0.000942,0.000515,-0.000338,0.001524,0.001336
1,2012-02-02,-0.002092,-0.006366,-0.011932,0.003138,0.000748,0.000811,0.006962,0.003000,0.001030,...,0.000753,-0.003676,-0.000164,-0.001918,-0.000135,-0.001475,-0.000015,0.003826,-0.000887,-0.005655
2,2012-02-03,0.014475,0.009887,0.029574,0.023525,0.015008,0.014078,0.014515,0.011870,0.015099,...,-0.001951,-0.008611,0.000894,-0.003463,0.002967,-0.004396,-0.001229,0.000470,-0.000913,-0.002213
3,2012-02-06,-0.006248,-0.002361,0.008876,0.006999,0.006416,-0.000872,-0.005803,-0.002699,-0.004013,...,0.000204,-0.000902,-0.001292,-0.000107,-0.002188,0.000196,0.000471,-0.000594,-0.002922,-0.002013
4,2012-02-07,0.004467,0.004342,0.001791,-0.000797,-0.005993,0.003868,0.000615,0.000360,-0.000120,...,-0.000147,0.000419,0.005658,0.001121,-0.001103,-0.000137,-0.000722,0.000021,0.000957,0.003163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,2023-11-24,0.000923,0.002835,0.011163,0.005302,0.002929,0.002315,0.003730,0.001062,0.003371,...,-0.000812,0.001020,-0.000492,-0.000595,-0.001386,0.000571,-0.000162,0.000299,0.000070,0.000756
2974,2023-11-27,-0.002993,-0.000953,-0.002744,-0.009048,-0.003523,-0.004220,-0.005795,-0.002816,-0.011146,...,0.000273,-0.000774,0.002020,0.000627,0.000834,-0.002090,0.000296,-0.001020,-0.001993,-0.000143
2975,2023-11-28,0.000258,0.003018,0.000855,0.001698,-0.004790,0.002063,-0.003305,0.006325,0.003842,...,0.001374,-0.000415,-0.000975,-0.001937,0.001649,-0.001388,0.001145,-0.000078,-0.000023,0.001334
2976,2023-11-29,0.007256,0.007497,0.016340,0.000338,0.007425,0.008129,-0.002609,-0.007278,-0.000883,...,0.003837,-0.002529,-0.003112,-0.001284,0.004110,-0.010430,-0.000128,0.001801,-0.000534,-0.000666


# Factor Risk -----------------------------

In [22]:
# Antag: fct_ret_est er en Pandas DataFrame med kolonnerne 'date', 'data' og 'res'

# 1. Udtræk 'id'-kolonnen fra den nestede 'data'-kolonne
fct_ret_est = fct_ret_est.copy()  # Arbejd på en kopi for at undgå at ændre originalen
fct_ret_est['id'] = fct_ret_est['data'].apply(lambda df: df['id'].tolist())
# Her antages det, at hver celle i 'data' er en DataFrame med en kolonne 'id'

# 2. Vælg kun de relevante kolonner
spec_risk = fct_ret_est[['id', 'date', 'res']].copy()

# 3. Kombiner de to list-kolonner (id og res) pr. række, så de bliver parret elementvis
# Det sikrer, at vi bevarer den korrekte parring mellem id og residual for hver række.
spec_risk['id_res'] = spec_risk.apply(lambda row: list(zip(row['id'], row['res'])), axis=1)

# 4. Eksploder (unnest) den nye kolonne 'id_res'
spec_risk = spec_risk.explode('id_res')

# 5. Split tuple-kolonnen 'id_res' op i to separate kolonner: 'id' og 'res'
spec_risk[['id', 'res']] = pd.DataFrame(spec_risk['id_res'].tolist(), index=spec_risk.index)

# 6. Fjern hjælpekolonnen 'id_res'
spec_risk = spec_risk.drop(columns=['id_res'])

# 7. Sorter resultaterne efter 'id' og 'date'
spec_risk = spec_risk.sort_values(['id', 'date']).reset_index(drop=True)

In [23]:
spec_risk

,id,date,res
0,10104,2012-02-01,0.013814
1,10104,2012-02-02,-0.003386
2,10104,2012-02-03,0.003454
3,10104,2012-02-06,-0.000870
4,10104,2012-02-07,0.002190
...,...,...,...
511495,92655,2023-11-24,0.000802
511496,92655,2023-11-27,-0.001250
511497,92655,2023-11-28,-0.004159
511498,92655,2023-11-29,0.010920


In [24]:
def unnest_spec_risk(fct_ret_est):
    """
    Udpakker de nestede 'id' og 'res'-lister i fct_ret_est.
    
    Forudsætninger:
      - fct_ret_est er en Pandas DataFrame med kolonnerne:
          - 'date': Datoen for gruppen
          - 'data': En nestet DataFrame, som indeholder en kolonne 'id'
          - 'res': En liste eller kolonne med residualer, der svarer til rækkerne i den nestede 'data'
    
    Funktionen gør følgende:
      1. Arbejder på en kopi af fct_ret_est.
      2. Ekstraherer 'id'-kolonnen fra hver nestet DataFrame i 'data' og gemmer den som en liste i en ny kolonne 'id'.
      3. Beholder kun kolonnerne 'id', 'date' og 'res'.
      4. Kombinerer de to list-kolonner ('id' og 'res') pr. række til en liste af tuples, så parringen bevares.
      5. "Exploder" (unnester) den kombinerede kolonne, så hver tuple bliver til en enkelt række.
      6. Splitter tuple-kolonnen op i to separate kolonner: 'id' og 'res'.
      7. Sorterer resultaterne efter 'id' og 'date' og returnerer den fladede DataFrame.
      
    Returnerer:
      En Pandas DataFrame med én række pr. observation, der indeholder kolonnerne 'id', 'date' og 'res'.
    """
    # Arbejd på en kopi af fct_ret_est
    df = fct_ret_est.copy()
    
    # 1. Udtræk 'id'-kolonnen fra den nestede 'data'-kolonne
    df['id'] = df['data'].apply(lambda nested_df: nested_df['id'].tolist())
    
    # 2. Vælg kun de relevante kolonner
    spec_risk = df[['id', 'date', 'res']].copy()
    
    # 3. Kombiner de to list-kolonner (id og res) pr. række til en liste af tuples
    spec_risk['id_res'] = spec_risk.apply(lambda row: list(zip(row['id'], row['res'])), axis=1)
    
    # 4. Eksploder (unnest) den nye kolonne 'id_res'
    spec_risk = spec_risk.explode('id_res')
    
    # 5. Split tuple-kolonnen 'id_res' op i to separate kolonner: 'id' og 'res'
    spec_risk[['id', 'res']] = pd.DataFrame(spec_risk['id_res'].tolist(), index=spec_risk.index)
    
    # 6. Fjern hjælpekolonnen 'id_res'
    spec_risk = spec_risk.drop(columns=['id_res'])
    
    # 7. Sorter resultaterne efter 'id' og 'date'
    spec_risk = spec_risk.sort_values(['id', 'date']).reset_index(drop=True)
    
    return spec_risk

In [25]:
spec_risk = unnest_spec_risk(fct_ret_est)

In [26]:
spec_risk

,id,date,res
0,10104,2012-02-01,0.013814
1,10104,2012-02-02,-0.003386
2,10104,2012-02-03,0.003454
3,10104,2012-02-06,-0.000870
4,10104,2012-02-07,0.002190
...,...,...,...
511495,92655,2023-11-24,0.000802
511496,92655,2023-11-27,-0.001250
511497,92655,2023-11-28,-0.004159
511498,92655,2023-11-29,0.010920


In [30]:
def calculate_ewma(spec_risk, settings):
    """
    Beregner EWMA-volatilitet per gruppe i en DataFrame.

    Args:
        spec_risk (pd.DataFrame): DataFrame med kolonnerne 'id' og 'res'.
        settings (dict): Dictionary med 'cov_set' indeholdende 'hl_stock_var' og 'initial_var_obs'.

    Returns:
        pd.DataFrame: Opdateret DataFrame med en ny kolonne 'res_vol'.

    Example:
    spec_risk_res_vol = calculate_ewma(spec_risk,settings)
    spec_risk_res_vol
    """

    lambda_val = 0.5 ** (1.0 / settings['cov_set']['hl_stock_var'])
    start_val = settings['cov_set']['initial_var_obs']

    def apply_ewma(group):
        group = group.copy()  # Sikrer, at vi ikke ændrer originalen
        group['res_vol'] = ewma.ewma_c(group['res'].values, lambda_val, start_val)
        return group
    
    # Anvend EWMA per gruppe og returner den opdaterede DataFrame
    return spec_risk.groupby('id', group_keys=False, as_index=False).apply(apply_ewma)


In [33]:
spec_risk_res_vol = calculate_ewma(spec_risk,settings)
spec_risk_res_vol

C:\Users\andre\AppData\Local\Temp\ipykernel_2460\135772750.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return spec_risk.groupby('id', group_keys=False, as_index=False).apply(apply_ewma)


,id,date,res,res_vol
0,10104,2012-02-01,0.013814,NaN
1,10104,2012-02-02,-0.003386,NaN
2,10104,2012-02-03,0.003454,NaN
3,10104,2012-02-06,-0.000870,NaN
4,10104,2012-02-07,0.002190,NaN
...,...,...,...,...
511495,92655,2023-11-24,0.000802,0.009110
511496,92655,2023-11-27,-0.001250,0.009085
511497,92655,2023-11-28,-0.004159,0.009061
511498,92655,2023-11-29,0.010920,0.009041


## ved ik om virker

In [69]:
def extract_specific_risk(spec_risk):
    """
    Denne funktion bearbejder en DataFrame med specific risk (spec_risk)
    og et sæt datoer (fct_dates) for at sikre, at vi kun beholder observationer,
    hvor vi har tilstrækkeligt med data (mindst 200 observationer ud af de seneste 252 dage)
    og udtrækker den seneste observation for hver aktie ved månedsslut.
    
    Parametre:
      - spec_risk: DataFrame med mindst kolonnerne 'id', 'date' og 'res_vol'
      - fct_dates: En sorteret Pandas Series eller liste med datoer (bruges til at beregne lag)
      
    Returnerer:
      - spec_risk_m: DataFrame med kolonnerne 'id', 'eom' (månedens sidste dag) og 'res_vol'
        for den seneste observation pr. aktie og måned.
    """
    # Sørg for, at fct_dates er en Pandas Series med datetime-objekter
    fct_dates = spec_risk['date'].sort_values()
    
    # 1. Opret en DataFrame med datoer og lag-værdi for 252 dage tidligere
    td_range = pd.DataFrame({'date': fct_dates})
    td_range['td_252d'] = td_range['date'].shift(252)
    
    # 2. Merge spec_risk med td_range på 'date'
    spec_risk = pd.merge(spec_risk, td_range, on='date', how='left')
    
    # 3. For hver aktie (id) beregnes en lag-værdi af 'date' med 200 perioder
    spec_risk['date_200d'] = spec_risk.groupby('id')['date'].shift(200)
    
    # 4. Filtrer: behold rækker, hvor date_200d >= td_252d og res_vol ikke er NaN
    spec_risk = spec_risk[(spec_risk['date_200d'] >= spec_risk['td_252d']) & (spec_risk['res_vol'].notnull())]
    
    # 5. Behold kun kolonnerne 'id', 'date' og 'res_vol'
    spec_risk = spec_risk[['id', 'date', 'res_vol']]
    
    # 6. Udtræk specific risk ved månedsslut:
    #    - Beregn 'eom_ret': sidste dag i måneden for hver dato.
    spec_risk['eom_ret'] = spec_risk['date'] + pd.offsets.MonthEnd(0)
    
    # 7. For hver kombination af id og eom_ret, beregn den seneste (max) dato
    spec_risk['max_date'] = spec_risk.groupby(['id', 'eom_ret'])['date'].transform('max')
    
    # 8. Filtrer til kun de rækker, hvor 'date' er lig med 'max_date'
    spec_risk_m = spec_risk[spec_risk['date'] == spec_risk['max_date']].copy()
    
    # 9. Behold kun de ønskede kolonner og omdøb 'eom_ret' til 'eom'
    spec_risk_m = spec_risk_m[['id', 'eom_ret', 'res_vol']].rename(columns={'eom_ret': 'eom'})
    
    # Returnér det endelige resultat
    return spec_risk_m

In [71]:
spec_risk_m = extract_specific_risk(spec_risk)

In [72]:
spec_risk_m

,id,eom,res_vol
3268,10104,2012-02-29,0.007931
3269,10104,2012-02-29,0.007931
3270,10104,2012-02-29,0.007931
3271,10104,2012-02-29,0.007931
3272,10104,2012-02-29,0.007931
...,...,...,...
87855563,92655,2023-11-30,0.006041
87855564,92655,2023-11-30,0.006041
87855565,92655,2023-11-30,0.006041
87855566,92655,2023-11-30,0.006041
